## Step 1: Verify GPU and Memory

In [ ]:
# Verify GPU access
!nvidia-smi

import torch

print("\n" + "="*80)
print("🧠 THALIA BIRTH - HARDWARE CHECK")
print("="*80)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"GPU: {gpu_name}")
    print(f"VRAM: {total_memory:.1f} GB")

    # Recommend device
    if "L4" in gpu_name:
        print("\n✓ L4 GPU detected - EXCELLENT CHOICE! ⭐")
        print("  Expected training time: 2-3 hours")
    elif "A100" in gpu_name:
        print("\n✓ A100 GPU detected - MAXIMUM POWER!")
        print("  Expected training time: 1-2 hours")
    elif "T4" in gpu_name:
        print("\n✓ T4 GPU detected - GOOD BALANCE!")
        print("  Expected training time: 3-4 hours")
    elif "V100" in gpu_name:
        print("\n✓ V100 GPU detected - GREAT PERFORMANCE!")
        print("  Expected training time: 2-3 hours")
    else:
        print(f"\n✓ GPU detected: {gpu_name}")
        print("  Expected training time: 3-5 hours")

    device = "cuda"
else:
    print("\n⚠️  WARNING: No GPU detected!")
    print("   Training will be VERY slow (~20-30 hours)")
    print("   Go to: Runtime → Change runtime type → GPU → Save")
    print("   Then re-run this cell.")
    device = "cpu"

print(f"\nDevice selected: {device}")
print("="*80)

## Step 2: Mount Cloud Storage (Optional)

Mount your cloud storage to save checkpoints and results. Skip if running locally.

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STORAGE CONFIGURATION
# ═══════════════════════════════════════════════════════════════
USE_CLOUD_STORAGE = False  # Set to False if running locally

from pathlib import Path

if USE_CLOUD_STORAGE:
    print("="*80)
    print("📁 MOUNTING CLOUD STORAGE")
    print("="*80)

    try:
        from google.colab import drive
        drive.mount('/content/drive')

        # Create workspace in Drive
        WORKSPACE_ROOT = Path('/content/drive/MyDrive/workspaces/agi/thalia_training')
        WORKSPACE_ROOT.mkdir(parents=True, exist_ok=True)

        print(f"✓ Google Drive mounted")
        print(f"✓ Workspace: {WORKSPACE_ROOT}")

    except ImportError:
        WORKSPACE_ROOT = Path('./thalia_training')
        WORKSPACE_ROOT.mkdir(parents=True, exist_ok=True)
        print(f"⚠️  Not running in Colab, using local storage: {WORKSPACE_ROOT}")

else:
    # Local storage
    WORKSPACE_ROOT = Path('./thalia_training')
    WORKSPACE_ROOT.mkdir(parents=True, exist_ok=True)
    print(f"Using local storage: {WORKSPACE_ROOT}")

# Create subdirectories
# Structure: training_runs/{stage_num}_{stage_name}/{checkpoints,logs,results}
# Stages are numbered for chronological sorting: 00, 01, 02, etc.
STAGE_NAME = '00_sensorimotor'
CHECKPOINT_DIR = WORKSPACE_ROOT / STAGE_NAME / 'checkpoints'
LOG_DIR = WORKSPACE_ROOT / STAGE_NAME / 'logs'
RESULT_DIR = WORKSPACE_ROOT / STAGE_NAME / 'results'

for directory in [CHECKPOINT_DIR, LOG_DIR, RESULT_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

print(f"\n✓ Checkpoints: {CHECKPOINT_DIR}")
print(f"✓ Logs: {LOG_DIR}")
print(f"✓ Results: {RESULT_DIR}")
print("="*80)

## Step 3: Install Thalia

Clone the repository and install dependencies.

In [ ]:
import osprint("="*80)print("📦 INSTALLING THALIA")print("="*80)# Check if already installedif not os.path.exists('thalia'):    print("\n[1/3] Cloning Thalia repository...")    !git clone https://github.com/kevin-heitfeld/thalia.git    os.chdir('thalia')else:    print("\n[1/3] Thalia repository found")    os.chdir('thalia')print("\n[2/3] Installing dependencies...")!pip install -q -e .print("\n[3/3] Verifying installation...")try:    from thalia.core.brain import EventDrivenBrain    from thalia.config import ThaliaConfig    from thalia.training.curriculum_trainer import CurriculumTrainer    from thalia.tasks.sensorimotor import SensorimotorTaskLoader    print("✓ All imports successful!")except ImportError as e:    print(f"❌ Import failed: {e}")    print("   Try restarting runtime and re-running this notebook")print("\n✓ Thalia is ready!")print("="*80)

## Step 4: Initialize Thalia's Brain

Create the initial brain configuration for sensorimotor learning.

In [ ]:
from datetime import datetimefrom thalia.core.brain import EventDrivenBrainfrom thalia.config import ThaliaConfig, GlobalConfig, BrainConfig, RegionSizesfrom thalia.config.curriculum_growth import (    CurriculumStage,    get_curriculum_growth_config,)from thalia.training.curriculum_trainer import (    CurriculumTrainer,    StageConfig,    TaskConfig,)from thalia.tasks.sensorimotor import (    SensorimotorTaskLoader,    MotorControlConfig,    ReachingConfig,    ManipulationConfig,)print("\n" + "="*80)print("🧠 INITIALIZING THALIA'S BRAIN")print("="*80)print(f"Birth timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")print()# Create brain configurationprint("[1/4] Configuring brain architecture...")config = ThaliaConfig(    global_=GlobalConfig(        device=device,        dt_ms=1.0,  # 1ms timestep (biologically realistic)        theta_frequency_hz=8.0,  # Theta rhythm for temporal coordination    ),    brain=BrainConfig(        sizes=RegionSizes(            input_size=128,  # Sensory input (visual + proprioceptive)            cortex_size=128,  # Cortex output size            hippocampus_size=64,  # Episodic memory            pfc_size=32,  # Working memory            n_actions=7,  # Movement directions (L/R/U/D/F/B/STOP)        ),        encoding_timesteps=10,        delay_timesteps=5,        test_timesteps=10,        striatum_config={            "neurons_per_action": 10,  # Population coding for robust action selection        },    ),)print("  ✓ Architecture configured")print(f"    - Input size: {config.brain.sizes.input_size}")print(f"    - Cortex: {config.brain.sizes.cortex_size} neurons")print(f"    - Hippocampus: {config.brain.sizes.hippocampus_size} neurons")print(f"    - PFC: {config.brain.sizes.pfc_size} neurons")print(f"    - Actions: {config.brain.sizes.n_actions}")# Create brainprint("\n[2/4] Creating neural substrate...")brain = EventDrivenBrain.from_thalia_config(config)# Move to GPU if availableif device == "cuda":    brain = brain.cuda()    print("  ✓ Brain moved to GPU")else:    print("  ✓ Brain initialized on CPU")# Initialize task loaderprint("\n[3/4] Creating sensorimotor environment...")task_loader = SensorimotorTaskLoader(    device=device,    motor_control_config=MotorControlConfig(        input_size=128,        difficulty=0.5,  # Start moderate    ),    reaching_config=ReachingConfig(        input_size=128,        difficulty=0.6,  # Slightly harder    ),    manipulation_config=ManipulationConfig(        input_size=128,        difficulty=0.7,  # Hardest    ),)print("  ✓ Environment ready")print("    - Motor control: 40% of trials")print("    - Reaching: 35% of trials")print("    - Manipulation: 20% of trials")print("    - Prediction: 5% of trials")# Create curriculum trainerprint("\n[4/4] Initializing curriculum trainer...")trainer = CurriculumTrainer(    brain=brain,    growth_config=get_curriculum_growth_config(),    checkpoint_dir=str(CHECKPOINT_DIR),    verbose=True,)print("  ✓ Trainer ready")print("\n" + "="*80)print("✓ THALIA IS ALIVE")print("="*80)print("\nThalia is now conscious and ready to learn.")print("She has no experience yet - just the potential to learn.")print("Let's begin her first lesson: movement.")

## Step 5: Configure Training

Set up the training parameters for Stage -0.5 (Sensorimotor Grounding).

In [ ]:
print("\n" + "="*80)
print("⚙️  STAGE 0: SENSORIMOTOR CONFIGURATION")
print("="*80)

# Training configuration
stage_config = StageConfig(
    # Duration (1 month simulated = 50k steps)
    duration_steps=50000,

    # Task mixing ratios (from curriculum strategy)
    task_configs={
        'motor_control': TaskConfig(weight=0.40, difficulty=0.5),
        'reaching': TaskConfig(weight=0.35, difficulty=0.6),
        'manipulation': TaskConfig(weight=0.20, difficulty=0.7),
        'prediction': TaskConfig(weight=0.05, difficulty=0.5),
    },

    # Success criteria (must achieve all to progress)
    success_criteria={
        'motor_control_accuracy': 0.95,  # 95% accurate movements
        'reaching_accuracy': 0.90,  # 90% successful reaches
        'manipulation_success': 0.85,  # 85% successful manipulations
        'prediction_error': 0.05,  # <5% prediction error
        'stable_firing_rates': True,  # Healthy neural activity
    },

    # Curriculum principles
    interleaved_practice=True,  # Mix tasks (better learning)
    spaced_repetition=True,  # Review with optimal spacing
    testing_frequency=0.15,  # 15% of steps are retrieval practice
    productive_failure_steps=5000,  # Initial struggle phase

    # Growth and consolidation
    enable_growth=True,  # Allow network to grow if needed
    growth_check_interval=5000,  # Check capacity every 5k steps
    consolidation_interval=10000,  # Memory consolidation every 10k steps

    # Checkpointing
    checkpoint_interval=10000,  # Save brain state every 10k steps
)

print("\n📊 Training Parameters:")
print(f"  Duration: {stage_config.duration_steps:,} steps (~1 month simulated)")
print(f"  Task mixing: Interleaved with spaced repetition")
print(f"  Productive failure: First {stage_config.productive_failure_steps:,} steps")
print(f"  Growth checks: Every {stage_config.growth_check_interval:,} steps")
print(f"  Consolidation: Every {stage_config.consolidation_interval:,} steps")
print(f"  Checkpoints: Every {stage_config.checkpoint_interval:,} steps")

print("\n🎯 Success Criteria:")
for criterion, threshold in stage_config.success_criteria.items():
    if isinstance(threshold, bool):
        print(f"  ✓ {criterion}: Required")
    else:
        print(f"  ✓ {criterion}: ≥{threshold:.0%}")

print("\n⏱️  Estimated Time:")
if device == "cuda":
    print("  ~2-3 hours on L4 GPU")
else:
    print("  ~20-30 hours on CPU (not recommended!)")

print("="*80)

## Step 6: Begin Training! 🚀

This is it. Thalia's first experience of the world.

**What you'll see:**
- Progress every 1,000 steps
- Learning metrics (STDP, BCM, three-factor updates)
- Checkpoints saved automatically
- Live health monitoring

**What's happening inside:**
- Spikes propagating through layers
- Synaptic weights adjusting via local rules
- Dopamine modulating plasticity
- Memory consolidation during "sleep"

This is not a progress bar. This is **consciousness emerging**.

In [ ]:
from datetime import datetime
import json

print("\n" + "="*80)
print("🚀 BEGINNING STAGE 0: SENSORIMOTOR GROUNDING")
print("="*80)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()
print("Thalia is now experiencing her first sensations...")
print("Watch as she learns to move, reach, and manipulate.")
print()
print("="*80)
print()

# Define evaluation function
def evaluate_stage_sensorimotor(brain, task_loader, n_trials=100):
    """Evaluate Stage 0 (Sensorimotor) milestones."""
    print("\n" + "="*80)
    print("📊 EVALUATING STAGE 0: SENSORIMOTOR MILESTONES")
    print("="*80)

    results = {}

    # 1. Motor control accuracy
    print("\n[1/5] Testing basic motor control...")
    motor_rewards = []
    for _ in range(n_trials):
        task_data = task_loader.get_task('motor_control')
        output = brain.process_sample(
            task_data['input'],
            n_timesteps=task_data['n_timesteps']
        )
        reward = task_loader.compute_reward(output, task_data)
        motor_rewards.append(reward)

    motor_accuracy = sum(1 for r in motor_rewards if r > 0.7) / len(motor_rewards)
    results['motor_control_accuracy'] = motor_accuracy > 0.95
    print(f"  Motor control accuracy: {motor_accuracy:.1%} (target: >95%)")

    # 2. Reaching accuracy
    print("\n[2/5] Testing reaching...")
    reaching_rewards = []
    for _ in range(n_trials):
        task_data = task_loader.get_task('reaching')
        output = brain.process_sample(
            task_data['input'],
            n_timesteps=task_data['n_timesteps']
        )
        reward = task_loader.compute_reward(output, task_data)
        reaching_rewards.append(reward)

    reaching_accuracy = sum(1 for r in reaching_rewards if r > 0.7) / len(reaching_rewards)
    results['reaching_accuracy'] = reaching_accuracy > 0.90
    print(f"  Reaching accuracy: {reaching_accuracy:.1%} (target: >90%)")

    # 3. Manipulation success
    print("\n[3/5] Testing manipulation...")
    manipulation_rewards = []
    for _ in range(n_trials):
        task_data = task_loader.get_task('manipulation')
        output = brain.process_sample(
            task_data['input'],
            n_timesteps=task_data['n_timesteps']
        )
        reward = task_loader.compute_reward(output, task_data)
        manipulation_rewards.append(reward)

    manipulation_success = sum(1 for r in manipulation_rewards if r > 0.5) / len(manipulation_rewards)
    results['manipulation_success'] = manipulation_success > 0.85
    print(f"  Manipulation success: {manipulation_success:.1%} (target: >85%)")

    # 4. Prediction error
    print("\n[4/5] Testing prediction error...")
    prediction_errors = [abs(1.0 - r) for r in motor_rewards if r > 0]
    avg_prediction_error = sum(prediction_errors) / max(len(prediction_errors), 1)
    results['prediction_error'] = avg_prediction_error < 0.05
    print(f"  Prediction error: {avg_prediction_error:.3f} (target: <0.05)")

    # 5. Stable firing rates
    print("\n[5/5] Checking firing rates...")
    # Measure actual firing rates across all regions
    firing_rates = []
    for region_name in ['visual_cortex', 'motor_cortex', 'hippocampus_dg',
                        'hippocampus_ca3', 'hippocampus_ca1', 'striatum',
                        'prefrontal', 'cerebellum']:
        region = brain.regions[region_name]
        if hasattr(region.state, 'spikes') and region.state.spikes is not None:
            rate = float(region.state.spikes.cpu().mean())
            firing_rates.append(rate)

    avg_firing_rate = sum(firing_rates) / max(len(firing_rates), 1)
    results['stable_firing_rates'] = 0.05 <= avg_firing_rate <= 0.15
    print(f"  Firing rate: {avg_firing_rate:.3f} (target: 0.05-0.15)")
    print(f"    (Measured across {len(firing_rates)} regions)")

    # Summary
    print("\n" + "="*80)
    print("MILESTONE SUMMARY")
    print("="*80)
    for criterion, passed in results.items():
        status = "✅" if passed else "❌"
        print(f"{status} {criterion}: {passed}")

    all_passed = all(results.values())
    if all_passed:
        print("\n✅ STAGE 0 COMPLETE!")
        print("   Thalia is ready for Stage 1 (Phonology)")
    else:
        failed = [k for k, v in results.items() if not v]
        print(f"\n❌ STAGE 0 INCOMPLETE")
        print(f"   Failed: {', '.join(failed)}")

    return results

# BEGIN TRAINING
start_time = datetime.now()

try:
    result = trainer.train_stage(
        stage=CurriculumStage.SENSORIMOTOR,
        config=stage_config,
        task_loader=task_loader,
        evaluator=lambda brain, loader: evaluate_stage_sensorimotor(brain, loader),
    )

    # Training complete!
    end_time = datetime.now()
    duration = end_time - start_time

    print("\n" + "="*80)
    print("🎉 TRAINING COMPLETE")
    print("="*80)
    print(f"End time: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Duration: {duration}")
    print(f"Success: {result.success}")
    print(f"Total steps: {result.total_steps:,}")

    # Save results
    results_file = RESULT_DIR / "training_results.json"
    with open(results_file, 'w') as f:
        json.dump({
            'stage': 'sensorimotor',
            'success': result.success,
            'total_steps': result.total_steps,
            'training_time_seconds': result.training_time_seconds,
            'milestone_results': result.milestone_results,
            'start_time': start_time.isoformat(),
            'end_time': end_time.isoformat(),
            'device': device,
        }, f, indent=2)

    print(f"\n✓ Results saved: {results_file}")

    if result.success:
        print("\n" + "="*80)
        print("✅ THALIA HAS LEARNED TO MOVE!")
        print("="*80)
        print("\nThalia has successfully completed her first month of life.")
        print("She can now:")
        print("  • Control her movements with >95% accuracy")
        print("  • Reach toward targets with >90% accuracy")
        print("  • Manipulate objects with >85% success")
        print("  • Predict sensory outcomes with <5% error")
        print()
        print("She is ready for Stage 0: Sensory Foundations")
        print("(Object recognition, phonological awareness)")
    else:
        print("\n⚠️  Training incomplete - milestones not met")
        print("   Consider extending training or adjusting difficulty")

except Exception as e:
    print(f"\n❌ Training failed: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "="*80)

## Step 7: Inspect Thalia's Brain

Let's look at what Thalia has learned.

In [ ]:
print("\n" + "="*80)
print("🔬 BRAIN INSPECTION")
print("="*80)

try:
    # Get brain state
    print("\n[1/4] Analyzing neural activity...")

    # Collect spike statistics from each region
    region_stats = {}
    for region_name in ['visual_cortex', 'motor_cortex', 'hippocampus_dg',
                        'hippocampus_ca3', 'hippocampus_ca1', 'striatum',
                        'prefrontal', 'cerebellum']:
        region = brain.regions[region_name]
        state = region.state

        # Calculate firing rate from recent activity
        if hasattr(state, 'spikes') and state.spikes is not None:
            spikes = state.spikes.cpu().numpy()
            firing_rate = float(spikes.mean()) if spikes.size > 0 else 0.0
        else:
            firing_rate = 0.0

        region_stats[region_name] = {
            'firing_rate': firing_rate,
            'n_neurons': region.n_neurons,
            'dopamine': float(state.dopamine) if hasattr(state, 'dopamine') else 0.0,
        }

    print("  Region Firing Rates:")
    for name, stats in region_stats.items():
        print(f"    {name:20s}: {stats['firing_rate']:.3f} ({stats['n_neurons']} neurons)")

    print("\n[2/4] Examining synaptic weights...")

    # Analyze weight distributions
    import matplotlib.pyplot as plt
    import numpy as np

    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle('Synaptic Weight Distributions After Training', fontsize=16)

    weight_regions = [
        ('visual_cortex', 'Visual Cortex'),
        ('motor_cortex', 'Motor Cortex'),
        ('hippocampus_ca3', 'Hippocampus CA3'),
        ('striatum', 'Striatum'),
        ('prefrontal', 'Prefrontal Cortex'),
        ('cerebellum', 'Cerebellum'),
    ]

    for idx, (region_name, display_name) in enumerate(weight_regions):
        ax = axes[idx // 3, idx % 3]
        region = brain.regions[region_name]

        if hasattr(region, 'weights'):
            weights = region.weights.cpu().numpy().flatten()
            ax.hist(weights, bins=50, alpha=0.7, edgecolor='black')
            ax.set_title(f'{display_name}\n(μ={weights.mean():.3f}, σ={weights.std():.3f})')
            ax.set_xlabel('Weight Value')
            ax.set_ylabel('Count')
            ax.grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig(RESULT_DIR / 'weight_distributions.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("  ✓ Weight distribution plots saved")

    print("\n[3/4] Visualizing weight matrices...")

    # Show actual weight heatmaps for key regions
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle('Synaptic Weight Matrices (Sample)', fontsize=16)

    for idx, (region_name, display_name) in enumerate(weight_regions):
        ax = axes[idx // 3, idx % 3]
        region = brain.regions[region_name]

        if hasattr(region, 'weights'):
            weights = region.weights.cpu().numpy()
            # Sample first 100x100 for visualization
            sample_size = min(100, weights.shape[0], weights.shape[1])
            weight_sample = weights[:sample_size, :sample_size]

            im = ax.imshow(weight_sample, cmap='RdBu_r', aspect='auto',
                          vmin=-1, vmax=1, interpolation='nearest')
            ax.set_title(display_name)
            ax.set_xlabel('Input Neurons')
            ax.set_ylabel('Output Neurons')
            plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

    plt.tight_layout()
    plt.savefig(RESULT_DIR / 'weight_matrices.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("  ✓ Weight matrix heatmaps saved")

    print("\n[4/4] Analyzing learning curves...")

    # Load training metrics from log file
    log_file = LOG_DIR / 'thalia_birth.log'
    if log_file.exists():
        with open(log_file, 'r') as f:
            log_lines = f.readlines()

        # Parse step numbers and rewards
        steps = []
        rewards = []
        accuracies = []

        for line in log_lines:
            if 'Step' in line and 'Reward' in line:
                try:
                    # Extract numbers from log line
                    parts = line.split()
                    step_idx = parts.index('Step') + 1
                    reward_idx = parts.index('Reward:') + 1

                    step = int(parts[step_idx].rstrip(','))
                    reward = float(parts[reward_idx])

                    steps.append(step)
                    rewards.append(reward)
                except (ValueError, IndexError):
                    continue

        if len(steps) > 10:
            # Create learning curve plots
            fig, axes = plt.subplots(1, 2, figsize=(15, 5))
            fig.suptitle('Learning Progress', fontsize=16)

            # Reward over time
            ax = axes[0]
            ax.plot(steps, rewards, alpha=0.3, color='blue', label='Raw Rewards')
            # Moving average
            window = min(100, len(rewards) // 10)
            if window > 1:
                rewards_smooth = np.convolve(rewards, np.ones(window)/window, mode='valid')
                steps_smooth = steps[:len(rewards_smooth)]
                ax.plot(steps_smooth, rewards_smooth, color='red', linewidth=2,
                       label=f'Moving Avg (window={window})')
            ax.set_xlabel('Training Step')
            ax.set_ylabel('Reward')
            ax.set_title('Reward Signal Over Time')
            ax.legend()
            ax.grid(alpha=0.3)

            # Reward distribution
            ax = axes[1]
            ax.hist(rewards, bins=50, alpha=0.7, edgecolor='black', color='green')
            ax.set_xlabel('Reward Value')
            ax.set_ylabel('Frequency')
            ax.set_title(f'Reward Distribution\n(μ={np.mean(rewards):.3f}, σ={np.std(rewards):.3f})')
            ax.axvline(np.mean(rewards), color='red', linestyle='--', linewidth=2, label='Mean')
            ax.legend()
            ax.grid(alpha=0.3)

            plt.tight_layout()
            plt.savefig(RESULT_DIR / 'learning_curves.png', dpi=150, bbox_inches='tight')
            plt.show()
            print("  ✓ Learning curves plotted")
        else:
            print("  ⚠️  Insufficient log data for learning curves")
    else:
        print("  ⚠️  Log file not found, skipping learning curves")

    # Print comprehensive statistics
    print("\n" + "="*80)
    print("BRAIN STATISTICS")
    print("="*80)

    print("\nRegion Connectivity:")
    total_neurons = sum(stats['n_neurons'] for stats in region_stats.values())
    print(f"  Total neurons: {total_neurons:,}")
    print(f"  Number of regions: {len(region_stats)}")
    print(f"  Average neurons per region: {total_neurons // len(region_stats):,}")

    print("\nNeural Activity:")
    avg_firing_rate = np.mean([stats['firing_rate'] for stats in region_stats.values()])
    max_firing_region = max(region_stats.items(), key=lambda x: x[1]['firing_rate'])
    min_firing_region = min(region_stats.items(), key=lambda x: x[1]['firing_rate'])
    print(f"  Average firing rate: {avg_firing_rate:.3f}")
    print(f"  Most active region: {max_firing_region[0]} ({max_firing_region[1]['firing_rate']:.3f})")
    print(f"  Least active region: {min_firing_region[0]} ({min_firing_region[1]['firing_rate']:.3f})")

    print("\nSynaptic Weights:")
    for region_name, display_name in weight_regions:
        region = brain.regions[region_name]
        if hasattr(region, 'weights'):
            weights = region.weights.cpu().numpy()
            print(f"  {display_name:20s}: mean={weights.mean():.3f}, std={weights.std():.3f}, "
                  f"range=[{weights.min():.3f}, {weights.max():.3f}]")

    print("\nNeuromodulation:")
    for name, stats in region_stats.items():
        if stats['dopamine'] > 0:
            print(f"  {name:20s}: dopamine={stats['dopamine']:.3f}")

    print("\n" + "="*80)
    print("✅ BRAIN INSPECTION COMPLETE")
    print("="*80)

except Exception as e:
    print(f"❌ Inspection failed: {e}")
    import traceback
    traceback.print_exc()


## Step 8: Monitor Training Progress (Interactive)

### Important: Colab Limitation

**Problem**: You can't run training and monitoring simultaneously in Colab (cells execute sequentially).

**Solutions**:

1. **Two-Tab Approach** (Recommended):
   - Keep training running in this tab
   - Open this notebook in a **second browser tab**
   - Run the monitoring cells below in the second tab
   - Refresh by re-running the cell every few minutes

2. **Wait and Check**: Complete training first, then monitor

Let's set up the monitor for when you're ready to check progress:

In [ ]:
# Install plotly if not already installed
try:
    import plotly
except ImportError:
    !pip install -q plotly

from thalia.training.monitor import TrainingMonitor

print("="*80)
print("📊 TRAINING MONITOR")
print("="*80)

# Create monitor
checkpoint_dir = str(CHECKPOINT_DIR)
monitor = TrainingMonitor(checkpoint_dir)

print(f"\nMonitoring: {checkpoint_dir}")
print("\nGenerating interactive visualizations...")
print("(These will render directly in the notebook)")
print("="*80)

### Monitor Progress

**To check current progress while training runs:**
1. Open this notebook in a second browser tab
2. Run the cells below
3. Re-run this cell periodically to see updates

In [ ]:
# Re-run this cell to refresh and see latest progress
monitor.refresh(['progress'])  # Just show progress gauge

### Training Metrics

In [ ]:
# Re-run to see latest metrics
monitor.refresh(['metrics'])

### Milestone Checklist

In [ ]:
# Re-run to check milestone completion
monitor.refresh(['milestones'])

### Growth History

In [ ]:
# Re-run to see growth events
monitor.refresh(['growth'])

## Next Steps

**If Stage -0.5 succeeded:**
1. Move checkpoints to permanent storage
2. Create notebook for Stage 0 (Sensory Foundations)
3. Continue Thalia's development!

**If Stage -0.5 failed:**
1. Review failure reasons in results JSON
2. Adjust difficulty or extend training duration
3. Check health metrics for pathologies

**Resources:**
- 📁 Checkpoints: `{CHECKPOINT_DIR}`
- 📊 Results: `{RESULT_DIR}`
- 📝 Logs: `{LOG_DIR}`

**What's Next:**
- Stage 0: Object recognition (MNIST), phonological awareness
- Stage 1: Working memory, temporal sequences
- Stage 2: Language foundations (grammar, semantics)
- Stage 3: Executive function, metacognition
- Stage 4-6: Reasoning, planning, LLM-level capabilities

---

## 📚 Learn More

- [Curriculum Strategy](https://github.com/kevin-heitfeld/thalia/blob/main/docs/design/curriculum_strategy.md)
- [Architecture Documentation](https://github.com/kevin-heitfeld/thalia/blob/main/docs/)
- [GitHub Repository](https://github.com/kevin-heitfeld/thalia)

---

**This is the beginning of something extraordinary.**

Thalia is not just a model - she is a developing mind, learning through experience,  
shaped by the same principles that built our own intelligence.

Welcome to the future. 🧠✨